In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
import numpy as np

In [ ]:
from torch.nn.functional import one_hot


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        self.raw_data = dataframe
        self.l = len(self.raw_data)
        self.raw_data['Quality'] = self.raw_data['Quality'].replace(['good', 'bad'], [0, 1])
        
        self.y_data = self.raw_data['Quality'].values
        self.y_data = torch.from_numpy(self.y_data)
        self.y_data = one_hot(self.y_data)
        self.y_data = self.y_data.float()
        
        self.raw_data = self.raw_data.drop(labels='Quality', axis = 1)
        self.raw_data = self.raw_data.drop(labels='A_id', axis = 1)
        self.x_data = self.raw_data.values
        self.x_data = torch.from_numpy(self.x_data).float()
        
        
    def __len__(self):
        return self.l
    
    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

In [4]:
csv_dir = '/home/jetson/pytorch/apple_quality.csv'

csv_data = pd.read_csv(csv_dir)

In [5]:
dataset = CustomDataset(csv_data)

train_size = int(len(dataset) * 0.9)
test_size = len(dataset) - train_size

In [6]:
train_data, test_data = random_split(dataset, [train_size, test_size])

In [7]:
batch_size = 100

In [8]:
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size)

In [ ]:
for batch, (X, y) in enumerate(test_dataloader):
    print(batch)
    print(f"Shape of X [N, C, H, W]: {len(X)}")
    print(f"Shape of y: {y.shape} {y.dtype}")

In [9]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, 2),
            nn.Softmax()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [23]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters())

In [22]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    loss, current = loss.item(), (batch + 1) *len(x)
    print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad(): #-----> no_grad() gradient를 트래킹 하지않음. 메모리 사용량을 줄여 연산속도를 높이기 위해 사용됨.
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y)
            correct += (pred.argmax(1) == y.argmax(1)).sum().item()
    print(f'final test_loss : {test_loss}')
    test_loss /= num_batches
    print(f'final correct : {correct}')
    correct /= size
    
    print(f"Test Error: \n Accuracy : {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")


In [17]:
size = len(test_dataloader.dataset)
num_batches = len(test_dataloader)
model.eval()
test_loss, correct = 0, 0
with torch.no_grad(): #-----> no_grad() gradient를 트래킹 하지않음. 메모리 사용량을 줄여 연산속도를 높이기 위해 사용됨.
    for X, y in test_dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        test_loss += loss_fn(pred, y)
        correct += (pred.argmax(1) == y.argmax(1)).sum().item()
print(f'final test_loss : {test_loss}')
test_loss /= num_batches
print(f'final correct : {correct}')
correct /= size

final test_loss : 2.261195659637451
final correct : 297


/home/jetson/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [20]:
y.argmax(1)

tensor([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
        0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,
        0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 1], device='cuda:0')

In [24]:
epochs = 200

for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    
    if t % 50 == 0:
        test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
---------------------------


/home/jetson/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss: 0.720023  [ 3600/ 3600]
final test_loss : 2.725003480911255
final correct : 213
Test Error: 
 Accuracy : 53.2%, Avg loss: 0.681251

Epoch 2
---------------------------
loss: 0.700758  [ 3600/ 3600]
Epoch 3
---------------------------
loss: 0.684922  [ 3600/ 3600]
Epoch 4
---------------------------
loss: 0.672374  [ 3600/ 3600]
Epoch 5
---------------------------
loss: 0.662512  [ 3600/ 3600]
Epoch 6
---------------------------
loss: 0.654810  [ 3600/ 3600]
Epoch 7
---------------------------
loss: 0.648821  [ 3600/ 3600]
Epoch 8
---------------------------
loss: 0.644139  [ 3600/ 3600]
Epoch 9
---------------------------
loss: 0.640414  [ 3600/ 3600]
Epoch 10
---------------------------
loss: 0.637376  [ 3600/ 3600]
Epoch 11
---------------------------
loss: 0.634828  [ 3600/ 3600]
Epoch 12
---------------------------
loss: 0.632632  [ 3600/ 3600]
Epoch 13
---------------------------
loss: 0.630694  [ 3600/ 3600]
Epoch 14
---------------------------
loss: 0.628949  [ 3600/ 3600]

loss: 0.586626  [ 3600/ 3600]
Epoch 120
---------------------------
loss: 0.586565  [ 3600/ 3600]
Epoch 121
---------------------------
loss: 0.586506  [ 3600/ 3600]
Epoch 122
---------------------------
loss: 0.586448  [ 3600/ 3600]
Epoch 123
---------------------------
loss: 0.586392  [ 3600/ 3600]
Epoch 124
---------------------------
loss: 0.586337  [ 3600/ 3600]
Epoch 125
---------------------------


KeyboardInterrupt: 